In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import gc

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv


In [5]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
price = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')
calender =pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
sales = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv")
price = reduce_mem_usage(price)
calender = reduce_mem_usage(calender)
sales = reduce_mem_usage(sales)
pred_cols =[f'd_{i}' for i in range(1942, 1970)]
test_limit = 1913 #> 1913
valid_limit = 1941 # >1941
for col in pred_cols:
    sales[col] = 0
# calender.head()
calender.fillna('Unknown', inplace=True)
calender_cat = ['event_name_1','event_type_1','event_name_2','event_type_2']
for col in calender_cat:
    en = preprocessing.LabelEncoder()
    calender[col] = en.fit_transform(calender[col])

Mem. usage decreased to 130.48 Mb (37.5% reduction)
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 96.13 Mb (78.8% reduction)


In [7]:
import datetime
import calendar

def week_of_month(dt):
    #tgtdate = pd.to_datetime(dt)
    #tgtdate = tgtdate.to_datetime()
    tgtdate =pd.to_datetime(dt)

    days_this_month = calendar.mdays[tgtdate.month]
    for i in range(1, days_this_month):
        d = datetime.datetime(tgtdate.year, tgtdate.month, i)
        if d.day - d.weekday() > 0:
            startdate = d
            break
    # now we canuse the modulo 7 appraoch
    return (tgtdate - startdate).days //7 + 1

In [8]:
# New cols defore melt
# Week in month
# Week in year
# day

calender['day'] = calender['d'].apply(lambda x : x.replace('d_',"")).astype(int)
calender['week'] = calender['date'].apply(week_of_month).astype(int)
calender['yr_week'] = calender['date'].apply(lambda x: pd.to_datetime(x).isocalendar()[1]).astype(int)
#calender['d'].value_counts()
date_cols =[f'd_{i}' for i in range(900, 1970)]
sales_cols = ['id','item_id','dept_id','cat_id','store_id','state_id']

In [9]:
df = pd.melt(sales,
             id_vars = sales_cols,
             value_vars = date_cols,
             var_name = "d",
             value_name = "sales")
df = df.merge(calender, on ="d",copy = False)
df = df.merge(price, how ="left", on =["store_id", "item_id", "wm_yr_wk"],copy = False)
df['sell_price']=df.groupby('id')['sell_price'].apply(lambda x:x.fillna(x.mean()))

In [10]:
ca_index = df[df['state_id']== "CA"].index
tx_index = df[df['state_id']== "TX"].index
wi_index = df[df['state_id']== "WI"].index

df.loc[ca_index, 'snap'] = df.loc[ca_index]['snap_CA'].astype(int)
df.loc[tx_index, 'snap'] = df.loc[tx_index]['snap_TX'].astype(int)
df.loc[wi_index, 'snap'] = df.loc[wi_index]['snap_WI'].astype(int)

df['snap']= df['snap'].astype(int)
df.drop(columns=['snap_CA','snap_TX','snap_WI'], inplace = True)
df = reduce_mem_usage(df)

Mem. usage decreased to 4360.18 Mb (30.3% reduction)


In [11]:
# del price, sales, calender
del ca_index, tx_index,wi_index
gc.collect()

20

In [12]:
# Lag - Sales before a week
df['lag_7'] = df[['id','sales']].groupby('id')['sales'].shift(7)
# Lag 30 - Sales before a month
df['lag_30'] = df[['id','sales']].groupby('id')['sales'].shift(30)
# rolling mean sales of a window size of 7 over column lag_7
df["roll_7"] = df[["id", 'lag_7']].groupby("id")['lag_7'].transform(lambda x : x.rolling(7).mean())
#rolling mean sales of a window size of 30 over column lag_30
df["roll_30"] = df[["id", 'lag_30']].groupby("id")['lag_30'].transform(lambda x : x.rolling(30).mean())

In [14]:
val_cols =[f'd_{i}' for i in range(1914,1942 ) ]
submission = sales[['id',*val_cols]]
# sub =  pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sample_submission.csv")
submission['id'] = submission['id'].apply(lambda x: x.replace('evaluation','validation'))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [15]:
del price, sales, calender
gc.collect()

742

In [16]:
df.dropna(inplace= True)

In [17]:
group_cols =['item_id','store_id','wday','snap']
df_grouped = df[[*group_cols, 'sales']].groupby(by=group_cols).mean().reset_index()
df_grouped.rename(columns={'sales':'mean_sales'}, inplace=True)
df_grouped.head()

,item_id,store_id,wday,snap,mean_sales
0,FOODS_1_001,CA_1,1,0,0.770000
1,FOODS_1_001,CA_1,1,1,0.888889
2,FOODS_1_001,CA_1,2,0,0.855670
3,FOODS_1_001,CA_1,2,1,0.541667
4,FOODS_1_001,CA_1,3,0,0.567010


In [18]:
df = df.merge(df_grouped, how="left",on =group_cols, copy= False)


In [19]:
cat_cols = ['dept_id','cat_id','store_id','state_id','item_id']
for col in cat_cols:
    en = preprocessing.LabelEncoder()
    df[col]= en.fit_transform(df[col])
del cat_cols
gc.collect()

20

In [20]:
cols_drop = [ 'd', 'date', 'wm_yr_wk', 'weekday']
model_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id','wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'day',
       'week', 'yr_week', 'sell_price', 'snap', 'lag_7', 'lag_30', 'roll_7',
       'roll_30', 'mean_sales']
target ="sales"

In [21]:
df.drop(columns=cols_drop, inplace= True)
df = reduce_mem_usage(df)
del cols_drop
gc.collect()

Mem. usage decreased to 1440.47 Mb (56.6% reduction)


40

In [22]:
valid = df[df['day'] > valid_limit]
X = df[df['day'] < valid_limit][model_cols]
y = df[df['day'] < valid_limit][target]


In [30]:
# test_limit = 1913 #> 1913
# valid_limit = 1941 # >1941
test_2 = df[(df['day'] > test_limit) & (df['day'] <= valid_limit)]

In [31]:
del df
gc.collect()

140

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [33]:
del X, y
gc.collect()

40

In [34]:
import lightgbm as lgb
cat_feats = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'wday', 'month','year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'week']

In [35]:
# del X_train, X_test, y_train, y_test 
# gc.collect()

In [36]:
train_data = lgb.Dataset(X_train , label = y_train, categorical_feature=cat_feats, free_raw_data=False)
test_date = lgb.Dataset(X_test, label = y_test,categorical_feature=cat_feats,free_raw_data=False)
test_2_data = lgb.Dataset(test_2[model_cols], label = test_2[target],categorical_feature=cat_feats,free_raw_data=False)

In [40]:
# params = {
#         "objective" : "poisson",
#         "metric" :"rmse",
#         "force_row_wise" : True,
#         "learning_rate" : 0.075,
# #         "sub_feature" : 0.8,
#         "sub_row" : 0.75,
#         "bagging_freq" : 1,
#         "lambda_l2" : 0.1,
# #         "nthread" : 4
#         "metric": ["rmse"],
#     'verbosity': 1,
#     'num_iterations' : 1200,
#     'num_leaves': 128,
#     "min_data_in_leaf": 100,
# }

# lgb_params = {
#                     'boosting_type': 'gbdt',
#                     'objective': 'tweedie',
#                     'tweedie_variance_power': 1.1,
#                     'metric': 'rmse',
#                     'subsample': 0.5,
#                     'subsample_freq': 1,
#                     'learning_rate': 0.03,
#                     'num_leaves': 2**11-1,
#                     'min_data_in_leaf': 2**12-1,
#                     'feature_fraction': 0.5,
#                     'max_bin': 100,
#                     'n_estimators': 1400,
#                     'boost_from_average': False,
#                     'verbosity': 1,
#                 } 

params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 1200,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
}

In [43]:
model = lgb.train(params, train_data, valid_sets = [test_date,test_2_data], verbose_eval=50)

[50]	valid_0's rmse: 2.25626	valid_1's rmse: 2.09855
[100]	valid_0's rmse: 2.20522	valid_1's rmse: 2.0343
[150]	valid_0's rmse: 2.1875	valid_1's rmse: 2.01996
[200]	valid_0's rmse: 2.17203	valid_1's rmse: 2.00118
[250]	valid_0's rmse: 2.15876	valid_1's rmse: 1.99208
[300]	valid_0's rmse: 2.14764	valid_1's rmse: 1.98274
[350]	valid_0's rmse: 2.13803	valid_1's rmse: 1.96841
[400]	valid_0's rmse: 2.13028	valid_1's rmse: 1.96193
[450]	valid_0's rmse: 2.12314	valid_1's rmse: 1.95559
[500]	valid_0's rmse: 2.11812	valid_1's rmse: 1.95193
[600]	valid_0's rmse: 2.10874	valid_1's rmse: 1.94315
[650]	valid_0's rmse: 2.1053	valid_1's rmse: 1.93887
[700]	valid_0's rmse: 2.10161	valid_1's rmse: 1.93569
[750]	valid_0's rmse: 2.09809	valid_1's rmse: 1.93261
[800]	valid_0's rmse: 2.09412	valid_1's rmse: 1.927
[850]	valid_0's rmse: 2.0906	valid_1's rmse: 1.92432
[900]	valid_0's rmse: 2.08748	valid_1's rmse: 1.92027
[950]	valid_0's rmse: 2.08509	valid_1's rmse: 1.91776
[1000]	valid_0's rmse: 2.0823	valid

In [48]:
model.save_model("model.lgb")

In [49]:
X_valid = valid[model_cols]


In [50]:
y_pred = model.predict(X_valid)

In [63]:
valid['sales_pred'] = y_pred

In [64]:
valid.head()

,id,item_id,dept_id,cat_id,store_id,state_id,sales,wday,month,year,...,week,yr_week,sell_price,snap,lag_7,lag_30,roll_7,roll_30,mean_sales,sales_pred
29971670,HOBBIES_1_001_CA_1_evaluation,1437,3,1,0,0,0,3,5,2016,...,4,21,8.382812,0,0.0,1.0,1.286133,0.966797,0.515625,0.747070
29971671,HOBBIES_1_002_CA_1_evaluation,1438,3,1,0,0,0,3,5,2016,...,4,21,3.970703,0,1.0,0.0,0.571289,0.166626,0.206177,0.198249
29971672,HOBBIES_1_003_CA_1_evaluation,1439,3,1,0,0,0,3,5,2016,...,4,21,2.970703,0,0.0,1.0,0.571289,0.566895,0.226807,0.432690
29971673,HOBBIES_1_004_CA_1_evaluation,1440,3,1,0,0,0,3,5,2016,...,4,21,4.640625,0,4.0,7.0,1.286133,2.033203,1.608398,1.509291
29971674,HOBBIES_1_005_CA_1_evaluation,1441,3,1,0,0,0,3,5,2016,...,4,21,2.880859,0,2.0,2.0,1.142578,1.133789,0.814453,0.909875


In [66]:
valids_sub = valid.set_index(['id','day'])['sales_pred'].unstack().reset_index()

In [97]:
valids_sub.columns

Index(['id', 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952,
       1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964,
       1965, 1966, 1967, 1968, 1969],
      dtype='object', name='day')

In [99]:
f_cols = [f'F{i}' for i in range(1,29)]

In [101]:
valids_sub.columns = ['id',*f_cols]

In [105]:
submission.columns=['id',*f_cols]

In [106]:
submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,2,0,3,5,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_validation,0,1,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,1,1,0,2,1,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_validation,0,0,1,2,4,1,6,4,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_validation,1,0,2,3,1,0,3,2,3,...,0,0,0,2,1,0,0,2,1,0


In [112]:
final_sub = pd.concat([submission,valids_sub], axis =0)

In [116]:
sub.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [117]:
final_sub.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_validation,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_validation,0.0,0.0,1.0,2.0,4.0,1.0,6.0,4.0,0.0,...,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_validation,1.0,0.0,2.0,3.0,1.0,0.0,3.0,2.0,3.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0


In [118]:
final_sub.to_csv("submission.csv",index=False)

In [60]:
submission.head()

,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_validation,0,0,0,2,0,3,5,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_validation,0,1,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,1,1,0,2,1,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_validation,0,0,1,2,4,1,6,4,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_validation,1,0,2,3,1,0,3,2,3,...,0,0,0,2,1,0,0,2,1,0


In [ ]:
%%time

alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev



    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
#     te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
#                                                                           "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(f"submission_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission.csv",index=False)

In [130]:
new_valid = pd.DataFrame()
new_valid['id'] = valids_sub['id']
new_valid[f_cols]=(valids_sub[f_cols]*1.028 + valids_sub[f_cols]*1.028 +  valids_sub[f_cols]*1.028 )/3

In [134]:
final_sub_new = pd.concat([submission,new_valid], axis =0)

In [135]:
final_sub_new.to_csv("submission_1.csv",index=False)